In [58]:
import pandas as pd
import numpy as np
import ydf
import matplotlib.pyplot as plot

df = pd.read_csv("dam_data_imputed_lyndrassia.csv")


train_df = df.copy(True)
#train_df.columns
train_df = train_df.drop(columns=["ID", "Years Modified", "Assessment Date", "Assessment Date","Loss given failure - prop (Qm)","Loss given failure - liab (Qm)", "Loss given failure - BI (Qm)", "Total Loss Given Failure", "Expected Loss Value"])

test_df = train_df.copy(True)

for i in range(test_df.index.size):
    if (test_df.loc[i, "Inspection Frequency"] < 5):
        test_df.loc[i, "Inspection Frequency"] = 5

model = ydf.GradientBoostedTreesLearner(label="Probability of Failure", task=ydf.Task.REGRESSION).train(train_df)


Train model on 8406 examples
Model trained in 0:00:01.826431


In [31]:
model.describe()

In [ ]:
ydf_prediction = model.predict(test_df)
#print(df["Probability of Failure"])
for i in range(test_df.index.size):
    test_df.loc[i, "Probability of Failure"] = ydf_prediction[i]
#print(test_df["Probability of Failure"])

0       0.0998
1       0.1283
2       0.1394
3       0.0853
4       0.0837
         ...  
8401    0.1235
8402    0.1197
8403    0.0837
8404    0.0795
8405    0.0929
Name: Probability of Failure, Length: 8406, dtype: float64
0       0.091069
1       0.107199
2       0.113096
3       0.083276
4       0.084947
          ...   
8401    0.094658
8402    0.117122
8403    0.085243
8404    0.080698
8405    0.088422
Name: Probability of Failure, Length: 8406, dtype: float64


In [63]:
orig_x = train_df["Inspection Frequency"].to_numpy()
orig_y = train_df["Probability of Failure"].to_numpy()

new_x = test_df["Inspection Frequency"].to_numpy()
new_y = test_df["Probability of Failure"].to_numpy()

plot.scatter(orig_x, orig_y)
plot.scatter(new_x, new_y)

plot.show()


In [64]:
new_df = df.copy(True) 
new_df = new_df.replace(new_df["Probability of Failure"], pd.Series(new_y))

for i in range(new_df.index.size):
    new_df.loc[i, "Expected Loss Value"] = test_df.loc[i, "Probability of Failure"] * new_df.loc[i, "Total Loss Given Failure"] 

In [ ]:
compare_new = new_df["Expected Loss Value"].to_numpy()
compare_old = df["Expected Loss Value"].to_numpy()
plot.hist([compare_old, compare_new], bins=50,label=["Original", "After Frequency Change"])
plot.xlabel("Expected Loss Value, millions(£Q)")
plot.ylabel("Number of Dams")
plot.title(f"Changes to expected losses due to increased inspection frequency in {df.loc[0, "Region"]}")
plot.legend()
plot.show()


In [ ]:
new_df.to_csv(f"machine_learning_frequency_adjusted_{df.loc[0, "Region"]}.csv")
plot.savefig(f"frequency_adjusted_expected_loss__{df.loc[0, "Region"]}_histogram.png")

0       0.091069
1       0.107199
2       0.113096
3       0.083276
4       0.084947
          ...   
8401    0.094658
8402    0.117122
8403    0.085243
8404    0.080698
8405    0.088422
Length: 8406, dtype: float64